In [1]:
import pathlib
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
from torchsummary import summary
from matplotlib import rc

from siammot.modelling.track_head.EMM.attention import DeformableSiameseAttention

%matplotlib inline

%load_ext autoreload
%autoreload 2

font = {'family': 'Times New Roman', 'weight': 'bold', 'size': 12}
rc('font', **font)

In [4]:
batch_size = 8
n_channels = 128
n_query_key_channels = n_channels // 4
template_size = 15
sr_size = template_size * 2

template_features = torch.rand((batch_size, n_channels, template_size, template_size)).cuda()
sr_features = torch.rand((batch_size, n_channels, sr_size, sr_size)).cuda()

attention = DeformableSiameseAttention(n_channels, n_query_key_channels).cuda()
attentional_template_features, attentional_sr_features = attention(template_features, sr_features)
attentional_template_features.shape, attentional_sr_features.shape

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.